In [1]:
%matplotlib inline               
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('classified data without aqi.csv')

In [3]:
df.head()

,Avg PM2.5,Avg Max Temp,Avg Dew Temp,Avg Wind Speed,Avg Humidity
0,2,64.000000,40.000000,6.571429,60.500000
1,2,72.714286,54.857143,8.000000,73.071429
2,2,74.857143,51.142857,7.428571,65.214286
3,2,73.000000,51.285714,8.285714,67.428571
4,2,77.571429,58.285714,8.285714,71.785714


In [4]:
df.describe()

,Avg PM2.5,Avg Max Temp,Avg Dew Temp,Avg Wind Speed,Avg Humidity
count,449.000000,449.000000,449.000000,449.000000,449.000000
mean,0.496659,75.795737,49.840280,8.876869,60.937480
std,0.605490,7.451177,8.928992,2.263398,9.173827
min,0.000000,57.571429,21.571429,4.000000,25.571429
25%,0.000000,70.714286,43.571429,8.000000,54.714286
50%,0.000000,75.571429,51.000000,8.714286,62.142857
75%,1.000000,81.000000,57.428571,9.428571,67.785714
max,2.000000,95.857143,65.571429,37.857143,86.571429


In [5]:
df.shape

(449, 5)

In [6]:
X = df.drop('Avg PM2.5',axis=1)
y = np.asarray(df['Avg PM2.5'], dtype="|S6")

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
X_train.shape

(359, 4)

In [9]:
X_test.shape

(90, 4)

In [10]:
y_train.shape

(359,)

In [11]:
y_test.shape

(90,)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [14]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=1000)

parameter_space = {
    'hidden_layer_sizes': [(10,10,10), (50,50,50), (100,100,100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [17]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate': ['constant', 'adaptive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [18]:
# Best parameter set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.613 (+/-0.029) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.596 (+/-0.022) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}
0.574 (+/-0.054) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.593 (+/-0.026) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.599 (+/-0.013) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.577 (+/-0.021) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 100, 50), 'learning_rate': 'constant', 'solver': '

In [19]:
y_true, y_pred = y_test , clf.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
             precision    recall  f1-score   support

       b'0'       0.67      0.80      0.73        50
       b'1'       0.50      0.44      0.47        34
       b'2'       0.00      0.00      0.00         6

avg / total       0.56      0.61      0.58        90



C:\Users\theda\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
